In [16]:
from multiprocessing import Pool, TimeoutError

%run ProRocketRegression.ipynb
%run Constant.ipynb
%run Util.ipynb


class Optimizing_ProRocketRegression:
    
    def __init__(self,symbols):
        self.symbols = symbols
    
    
    def simulate(self,start_date,end_date,draw_pic=False):
        policy_name='rocket_break_v7_upgrade_honor_account_v0.3.json'

        regression = ProRocketRegression(self.symbols,start_date,end_date,policy_name)
        evaluation_df = regression.start_regression(True)

        return evaluation_df
    
    
    def case_regression_optimizing_stocks(self):        
        start_date = '2010-01-01'
        end_date = '2019-01-01'
        evaluation_df = self.simulate(start_date,end_date,True)
        
        return evaluation_df
    
    
    # 单支股票对于不同周期测试
    def case_compare_the_same_single_stock_in_two_decades(process_num,max_stocks):
        pool = Pool(processes=process_num)

        start_1 = '2000-01-01'
        end_1 = '2010-01-01'
        start_2 = '2010-01-01'
        end_2 = '2019-01-01'

        tasks = []
        for i in range(0,max_stocks):
            task = pool.apply_async(simulate, (i,i+1,start_1,end_1))
            tasks.append(task)
            task = pool.apply_async(simulate, (i,i+1,start_2,end_2))
            tasks.append(task)

        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        for task in tasks:
            try:
                one_evaluation = task.get()
                evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            except Exception as e:
                print(str(e))

        pool.close()
        pool.join()

        evaluation_df.to_csv(report_dir+'all_single_stocks_in_two_decades_evaluation_df.csv',index=False)

        return evaluation_df

    

    def filter_missing_symbols(evaluation_df):
        count_df = evaluation_df[['symbol','start_date','end_date','avg_year_return']].groupby(by='symbol')['start_date'].count()
        symbols = list(count_df[count_df==1].index.values)

        for symbol in symbols:
            evaluation_df = evaluation_df[evaluation_df['symbol']!=symbol]

        return evaluation_df


<IPython.core.display.Javascript object>

In [20]:
evaluation_df[display_columns]

,symbol,start_date,end_date,in_hand_asset_drawdown,avg_drawdown,float_profit_drawdown,avg_year_return,year_regular_return,mean_year_return,mean_month_return,profit_month_rate
1,all_symbols,2010-01-01,2019-01-01,9.92,8.873225,33.24,15.17,13.78,7.206667,0.575278,56.48
